# Introduction

https://mlbootcamp.ru/round/26/tasks/

Chinese is a wonderful language. Let's see what can we build to understand the meaning of the sentence and make a proper translation system.

Our main task is to receive a Russian translation for a Chinese text.

In [7]:
import pandas as pd
import numpy as np
import jieba
import pinyin.cedict
import pickle

from matplotlib import pyplot as plt

It's a random sentences from the BBC news channel.

In [19]:
text = "摆在唐宁街面前的是一系列重大决策：何时行动？解除哪些限制？如何控制病毒传播？如何权衡短期内挽救生命、长期内挽救经济和社会？"

Typical Chinese word consists of 2 or 3 characters. We can see that there are no spaces between words. Let's use the jieba library to split the sentence to words.

In [28]:
words = jieba.lcut(text)

In [48]:
words[:14]

['摆在',
 '唐宁街',
 '面前',
 '的',
 '是',
 '一系列',
 '重大',
 '决策',
 '：',
 '何时',
 '行动',
 '？',
 '解除',
 '哪些']

We can slightly understand what's it about by use the pinyin lib.

In [44]:
trans = []
for word in words:
    tr = pinyin.cedict.translate_word(word)
    if tr:
        trans.append(tr[0])
    else:
        trans.append("—")

In [47]:
trans[:14]

['—',
 'Downing Street (London)',
 'in front of',
 'aim',
 'variant of 是[shi4]',
 'a series of',
 'great',
 'strategic decision',
 '—',
 'when',
 'operation',
 '—',
 'to remove',
 'which ones?']

# Baseline

Naive approach is to translate with dictionary. We take the first Chinese-Russian dictionary thet we can find, clean it and use to translation.

In [24]:
zh_ru = pickle.load(open("zh_ru.pkl", "rb"))

In [52]:
len(zh_ru)

2616

In [53]:
def translate(word):
    if word in zh_ru:
        return zh_ru[word]
    else:
        return "пушкин"

In [56]:
for word in words:
    print(word, translate(word))

摆在 пушкин
唐宁街 пушкин
面前 пушкин
的 пушкин
是 ['представлять', 'представить']
一系列 ['серия']
重大 пушкин
决策 пушкин
： пушкин
何时 пушкин
行动 пушкин
？ пушкин
解除 пушкин
哪些 пушкин
限制 пушкин
？ пушкин
如何 ['как?']
控制 пушкин
病毒传播 пушкин
？ пушкин
如何 ['как?']
权衡 пушкин
短期内 пушкин
挽救 пушкин
生命 ['жизнь']
、 пушкин
长期 пушкин
内 пушкин
挽救 пушкин
经济 ['экономика']
和 ['с']
社会 пушкин
？ пушкин


In [68]:
import json
with open('data.json', 'w', encoding='utf8') as f:
    json.dump(zh_ru, f, ensure_ascii=False)
    
#     with io.open('filename', 'w', encoding='utf8') as json_file:
#     json.dump(u"ברי צקלה", json_file, ensure_ascii=False)

In [70]:
with open('data.json', 'r', encoding='utf8') as f:
    a = json.load(f)

In [71]:
a['而']

['а']

# Replace all non Chinese characters

In [7]:
# import re

# LHan = [[0x2E80, 0x2E99],    # Han # So  [26] CJK RADICAL REPEAT, CJK RADICAL RAP
#         [0x2E9B, 0x2EF3],    # Han # So  [89] CJK RADICAL CHOKE, CJK RADICAL C-SIMPLIFIED TURTLE
#         [0x2F00, 0x2FD5],    # Han # So [214] KANGXI RADICAL ONE, KANGXI RADICAL FLUTE
#         0x3005,              # Han # Lm       IDEOGRAPHIC ITERATION MARK
#         0x3007,              # Han # Nl       IDEOGRAPHIC NUMBER ZERO
#         [0x3021, 0x3029],    # Han # Nl   [9] HANGZHOU NUMERAL ONE, HANGZHOU NUMERAL NINE
#         [0x3038, 0x303A],    # Han # Nl   [3] HANGZHOU NUMERAL TEN, HANGZHOU NUMERAL THIRTY
#         0x303B,              # Han # Lm       VERTICAL IDEOGRAPHIC ITERATION MARK
#         [0x3400, 0x4DB5],    # Han # Lo [6582] CJK UNIFIED IDEOGRAPH-3400, CJK UNIFIED IDEOGRAPH-4DB5
#         [0x4E00, 0x9FC3],    # Han # Lo [20932] CJK UNIFIED IDEOGRAPH-4E00, CJK UNIFIED IDEOGRAPH-9FC3
#         [0xF900, 0xFA2D],    # Han # Lo [302] CJK COMPATIBILITY IDEOGRAPH-F900, CJK COMPATIBILITY IDEOGRAPH-FA2D
#         [0xFA30, 0xFA6A],    # Han # Lo  [59] CJK COMPATIBILITY IDEOGRAPH-FA30, CJK COMPATIBILITY IDEOGRAPH-FA6A
#         [0xFA70, 0xFAD9],    # Han # Lo [106] CJK COMPATIBILITY IDEOGRAPH-FA70, CJK COMPATIBILITY IDEOGRAPH-FAD9
#         [0x20000, 0x2A6D6],  # Han # Lo [42711] CJK UNIFIED IDEOGRAPH-20000, CJK UNIFIED IDEOGRAPH-2A6D6
#         [0x2F800, 0x2FA1D]]  # Han # Lo [542] CJK COMPATIBILITY IDEOGRAPH-2F800, CJK COMPATIBILITY IDEOGRAPH-2FA1D

# def build_re():
#     L = []
#     for i in LHan:
#         if isinstance(i, list):
#             f, t = i
#             try: 
#                 f = unichr(f)
#                 t = unichr(t)
#                 L.append('%s-%s' % (f, t))
#             except: 
#                 pass # A narrow python build, so can't use chars > 65535 without surrogate pairs!

#         else:
#             try:
#                 L.append(unichr(i))
#             except:
#                 pass

#     RE = '[%s]' % ''.join(L)
#     print('RE:', RE.encode('utf-8'))
#     return re.compile(RE, re.UNICODE)

# RE = build_re()
# print(RE.sub('', u'美国').encode('utf-8'))
# print(RE.sub('', u'blah').encode('utf-8'))

In [14]:
import re

pattern = re.compile(r'\s+')
s = re.sub(pattern, ' ', "Please \n don't \t hurt \x0b me.")

#s

# Merge books

In [8]:
import os
import glob
path = "D:\\ml\\ted-talks-transcript\\temp"

In [9]:
for f in [filenames for (dirpath, dirnames, filenames) in os.walk(path)]:
    types = ['\*_ru.txt', '\*_zh.txt']
    ru = []
    zh = []
    
    print(f)
    
    #print(path + f)
    files = glob.glob(path + types[0])        
    ru.extend(files)
    
    #print(path + f)
    files = glob.glob(path + types[1])        
    zh.extend(files)

['sent_ru_ext_ru.txt', 'sent_zh_ext_zh.txt']


In [10]:
ru

['D:\\ml\\ted-talks-transcript\\temp\\sent_ru_ext_ru.txt']

In [11]:
zh

['D:\\ml\\ted-talks-transcript\\temp\\sent_zh_ext_zh.txt']

In [12]:
for r,z in zip(ru,zh):
    print(r,z)

D:\ml\ted-talks-transcript\temp\sent_ru_ext_ru.txt D:\ml\ted-talks-transcript\temp\sent_zh_ext_zh.txt


In [15]:
pattern = re.compile(r'[a-zA-Z\s]+')
pattern2 = re.compile(r'[a-zA-Z]+')

# s = re.sub(pattern, ' ', "Please \n don't \t hurt \x0b me.")

with open("D:\\ml\\ted-talks-transcript\\temp\\sent_ext_ru.txt", mode="w", encoding="utf-8") as out_ru, \
     open("D:\\ml\\ted-talks-transcript\\temp\\sent_ext_zh.txt", mode="w", encoding="utf-8") as out_zh:
    for r,z in zip(ru,zh):
#         print(r,z)
        with open(r, mode="r", encoding="utf-8") as in_ru, open(z, mode="r", encoding="utf-8") as in_zh:
            lines_ru = in_ru.readlines()
            for line_ru in lines_ru:
                line_zh = in_zh.readline()
                out_ru.write(re.sub(pattern2, '', line_ru.strip()))
                out_zh.write(re.sub(pattern, '', line_zh.strip()))
                out_ru.write('\n')
                out_zh.write('\n')



# Merge news

In [5]:
import os
import glob
import re

path = r"D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated"

In [2]:
for f in [filenames for (dirpath, dirnames, filenames) in os.walk(path)]:
    types = ['\\ru*.txt', '\\zh*.txt']
    ru = []
    zh = []
    
    #print(f)
    
    #print(path + types[0])
    files = glob.glob(path + types[0])        
    ru.extend(files)
    
    print(files)
    
    #print(path + f)
    files = glob.glob(path + types[1])        
    zh.extend(files)

['D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_0.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_1.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_10.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_11.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_12.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_13.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_14.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_15.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_16.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_17.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_18.txt', 'D:\\ml\\umcorpus-v1\\UM-Corpus\\data\\Bilingual\\News\\translated\\ru.en_19.txt', 'D:\\

In [49]:
for r,z in zip(ru,zh):
    print(r,z)

D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_0.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_0.txt
D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_1.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_1.txt
D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_10.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_10.txt
D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_11.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_11.txt
D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_12.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_12.txt
D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_13.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_13.txt
D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\ru.en_14.txt D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\zh_14.txt
D:\ml\umcorpus-v1

In [6]:
pattern = re.compile(r'[a-zA-Z\s]+')
pattern2 = re.compile(r'[a-zA-Z]+')

# s = re.sub(pattern, ' ', "Please \n don't \t hurt \x0b me.")

with open(r"D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\news_ru.txt", mode="w", encoding="utf-8") as out_ru, \
     open(r"D:\ml\umcorpus-v1\UM-Corpus\data\Bilingual\News\translated\news_zh.txt", mode="w", encoding="utf-8") as out_zh:
    for r,z in zip(ru,zh):
#         print(r,z)
        with open(r, mode="r", encoding="utf-8") as in_ru, open(z, mode="r", encoding="utf-8") as in_zh:
            lines_ru = in_ru.readlines()
            for line_ru in lines_ru:
                line_zh = in_zh.readline()
                out_ru.write(re.sub(pattern2, '', line_ru.strip()))
                out_zh.write(re.sub(pattern, '', line_zh.strip()))
                out_ru.write('\n')
                out_zh.write('\n')